# TME 4 : Projet Detection de motifs

Nous allons développer des algorithmes pour chercher de motifs dans les données de ChipSeq de C. glabrata.
Pour commencer nous allons d'abord produire des données artificielles qui nous permettront de tester rapidement nos algorithmes. Ensuite nous allons chercher les motifs dans C. glabrata et analyser les résultats.

#### Binome : K Murali Sharane et Legendre--Despas Jeanne

## Partie A : Recherche de pattern (motifs) identiques

1\. Faites une fonction pour générer aléatoirement des séquences artificielles, puis générer t=10 séquences de n=41 nucléotides chacune. Toutes les lettres doivent être équiprobables pour la génération des séquences.

In [2]:
import random
import re
import operator
import copy

In [3]:
#import import_ipynb
#import TME2_LU3IN013

nuc = ('A', 'C', 'G', 'T')

t = 10 # nombre de sequences
n = 41 # longueur des sequences

#Pour un simple test
#t=3 #nombre de sequences
#n=8 #longueur des sequences

def generateRandomSequence(n, upper=True):
    """
    Génère une séquence nucléotidique aléatoire
    entrée n : longueur de la sequence
    entrée upper : bool, si True les nucléotides seront majuscule, False minuscule
    sortie sequence : une séquence nucléotidique aléatoire 
    """
    sequence = ""
    
    for i in range(n) : #On choisi n nucléotides aléatoirement dans ('A', 'C', 'G', 'T')
        if (upper) :
            sequence += random.choice(nuc)
        else :
            sequence += random.choice(nuc).lower()
   
    return sequence

def generateRandomSequences(n, t):
    """
    Génère plusieurs séquences nucléotidiques aléatoires 
    entrée n : longueur des séquences
    entrée t : nombre de séquences
    sortie séquences : une liste de t séquences nucléotidiques aléatoires de longueur n
    """
    return [generateRandomSequence(n) for k in range(t)]
    
listeSeqAlea = generateRandomSequences(n, t);

print("Génération de", t, "séquences aléatoires de", n,"nucléotides :\n")
for indice,seq in enumerate(listeSeqAlea) : 
    print("Séquence", indice+1 ,":",seq)


Génération de 10 séquences aléatoires de 41 nucléotides :

Séquence 1 : GATCGTATAATCAGAGGTTATGTCTCCTTTTAATTGTTCTC
Séquence 2 : AGGTGGACGAGTGCAAAGTTAAATCGGACCTTTTAAGAAGG
Séquence 3 : AGGTCAAGAGGAATCTTCCGAGCGCTCAGAGTACGTTGTGC
Séquence 4 : CGGCTTATTCAGGAACACCCAAGTGGGGGCCCTATATATAC
Séquence 5 : CCACCAGTGTGTATTGATAGCTTTGTACGTTCGAGTCAAGC
Séquence 6 : CCACGGCTCATTTTGCAGGCCTTCTGATGTGACCGTGAATA
Séquence 7 : CATTTCGACGTGTGCCGTTTCATAACGACTCCAGCGCACTA
Séquence 8 : GGATCGCTAATAGCGGGTCGTGTTCTTCGGGCTCACGTGTC
Séquence 9 : GGGCGCTTGGGTAGCGAGGATGACTGATCCCACTGGCCCCA
Séquence 10 : TGGTAAGCGATAGGGGCAGGACTTCCCCATGGTAGCCCACT


2\. Nous allons maintenant implanter un motif de taille `k`=9 dans une proportion ``f``=0.9 des séquences artificielles générées précédemment. Les motifs sont implantés à une position aléatoire (choisies uniformément le long de la séquence).

In [4]:
# taille des motifs k = 9
k = 9
# frequences d'occurences des motifs f = 0.9 
f = 0.9

def implantMotifs(motif, f, sequences):
    """
    Insérer un motif dans des positions aléatoires des séquences
    entrée motifs : motif qui va être implanté dans les séquences
    entrée f : fréquence d'implantation
    entrée séquences : liste de sequences
    sortie modified_sequences: liste de séquences ayant parfois le motif implanté
    """
    
    modified_sequences = []
    
    for sequence in sequences :
        
        if (random.random()<f) : #Le motif est implanté avec une fréquence de f
            
            #La position de début du motif implanté est choisies aléatoirement entre 0 et len(sequence)-len(motif)
            posDeb = random.randrange(len(sequence)-len(motif) +1)
            modified_sequences.append(sequence[:posDeb] + motif + sequence[posDeb+len(motif):])
            
        else : 
            modified_sequences.append(sequence)
    
    return modified_sequences


motif = generateRandomSequence(k, False) #On génère un motif aléatoire de longueur k = 9 

listeSeqMotif = implantMotifs(motif, f, listeSeqAlea ) #On implante le motif dans les séquences générées précédemment

print("Génération de", t, "séquences aléatoires de", n,"nucléotides avec le motif",motif,"implanté :\n")
for indice, sequence in enumerate(listeSeqMotif) :
    print("Séquence",indice+1,":",sequence)

Génération de 10 séquences aléatoires de 41 nucléotides avec le motif gaggtgata implanté :

Séquence 1 : GATCGgaggtgataAGGTTATGTCTCCTTTTAATTGTTCTC
Séquence 2 : AGGTGGACGAGTGCAAAGTTAAATCGGAgaggtgataAAGG
Séquence 3 : AGGTCgaggtgataCTTCCGAGCGCTCAGAGTACGTTGTGC
Séquence 4 : CgaggtgataAGGAACACCCAAGTGGGGGCCCTATATATAC
Séquence 5 : CCACCAGTGTGTAgaggtgataTTGTACGTTCGAGTCAAGC
Séquence 6 : CCACGGCTCATTTTGCgaggtgataGATGTGACCGTGAATA
Séquence 7 : CATTTCGACGTGTGCCGTTTCATAACGACTCCAGCGCACTA
Séquence 8 : GGATCGCTAATAGCGGGTCGTGTTCTTCGGGCgaggtgata
Séquence 9 : GGGCGCTTGGGTAGCGAGGATGACTGATCCCgaggtgataA
Séquence 10 : TGGTAgaggtgataGGCAGGACTTCCCCATGGTAGCCCACT


3\. Faites une fonction pour chercher les $m$ motifs de taille $k$ les plus fréquents dans l'ensemble des séquences. Tester cette fonction sur l'ensemble des séquences générées précédemment avec le motif implanté. Faite aussi une fonction qui affiche les $top$ motifs les plus fréquents. 

In [5]:
def searchMotifs(k, sequences):
    """
    Cherche les motifs de taille k dans un ensemble de séquences
    entrée k : taille du motif
    entrée séquences : liste de sequences
    sortie motifs: dictionnaire de motifs, clé = motif, valeur = fréquence d'observation
    >>>searchMotifs(3, ['TAAGTAA', 'TATAA', 'CTATC'])
    {'TAA': 3, 'AAG': 1, 'AGT': 1, 'GTA': 1, 'TAT': 2, 'ATA': 1, 'CTA': 1, 'ATC': 1}
    """
    
    motifs  = {}; # Dictionnaire des motifs associés à leur fréquence d'observation
    
    for sequence in sequences : # On parcourt la liste des séquences
        
        # Il y a len(sequence) - k + 1 positions posibles pour un motif de taille k
        # Les positions de début des motifs de taille k vont de 0 à len(sequence) - k
        
        for posDeb in range(len(sequence) - k + 1):
            
            #Le motif court de posDeb à posDeb + k - 1
            if (sequence[posDeb:posDeb+k] in motifs) :
                #On inclémente la valeur si le motif est déjà dans le dictionnaire 
                motifs[sequence[posDeb:posDeb+k]] += 1 
                
            else :
                #On ajoute le motif au dictionnaire avec une valeur de 1
                motifs[sequence[posDeb:posDeb+k]] = 1  

    return motifs

def getTopMotifs(motifs, top):
    """
    renvoyer les 'top' motifs les plus fréquents
    entrée motifs: dictionnaire de motifs, clé = motif, valeur = fréquence d'observation
    entrée top : le nombre des motifs les plus frequents à trouver
    sortie motifsfreq: dictionnaire contenant les 'top' motifs les plus fréquents, clé = motif, valeur = fréquence d'observation
    >>>getTopMotifs({'TAA': 3, 'AAG': 1, 'AGT': 1, 'GTA': 1, 'TAT': 2, 'ATA': 1, 'CTA': 1, 'ATC': 1}, 2)
    {'TAA': 3, 'TAT': 2}
    """
    
    #On récupère une liste de tuple (clef, valeur) éléments du dictionnaire motifs
    #triés dans l'odre décroissant de leur valeur
    listSortDict = sorted(motifs.items(), key=lambda x: x[1], reverse = True)
    
    #On retourne le dictionnaire formé des top premiers éléments de la liste 
    return dict([listSortDict[i] for i in range(top)])

top = 2
motifsFound = searchMotifs(k, listeSeqMotif)
#print (motifsFound)
topMotifs = getTopMotifs(motifsFound, top)
print("Les top =",top,"motifs de longueur k =",k,"les plus fréquents sont :\n")
print (topMotifs)

Les top = 2 motifs de longueur k = 9 les plus fréquents sont :

{'gaggtgata': 9, 'Cgaggtgat': 5}


4\. Quelle est la complexité de votre algorithme?

<font color='blue'>
La fonction sorted() est une fonction de tri fournie par Python. Les fonctions de tri proposées par Python sont implémentées avec l'algorithme du 'Timsort', un algorithme hybride en le tri par fusion et le tri par insertion. Cet algorithme a une complexité en $\theta$ ($n$log($n$)), n la longueur du dictionnaire. Ensuite on crée une nouvelle liste par compréhension avec les top premiers éléments de notre liste triée. Les opérations list[i] et list.append(elem) sont toutes les deux en $\theta$ (1), ce qui revient à $\theta$ (2*top). Enfin la fonction dict(list de (clef, valeur)) qui renvoie une dictionaire à partir d'une liste est en $\theta$ (len(list)) car on parcourt une fois toute la liste. ce qui revient ici à $\theta$ (top). 
  Ainsi la compléxité totale de notre algorithme est de $\theta$ ($n$log($n$)) + $\theta$ (2*top) + $\theta$ (top) soit $\theta$ ($n$log($n$)).
</font>

5\. Certains motifs sont reverse complémentaires. Implanter des motifs reverse complémentaires dans les séquences déjà générés et en suite chercher ces motifs. Attention: vous devez réutiliser les fonctions développés précédemment, pas besoin de définir des nouvelles fonctions. 

In [6]:
#Fonction du TME précédant.
def reversecompl(seq):
    seq_upper = seq.upper()
    compl = {'A': 'T', 'C': 'G', 'G': 'C', 'T':'A'}
    nvSeq = ""
    for i in range(len(seq_upper)-1,-1,-1) :
        nvSeq = nvSeq + compl[seq_upper[i]]
    
    return nvSeq

#Génération du motif reverse complémentaire correspondant au motif déjà généré précédemment.
motifRC = reversecompl(motif)
print("Le motif implanté :", motifRC,"\n")

#Implantation du motif RC dans la liste de séquences générées aléatoirement précédemment.
listeSeqMotifRC = implantMotifs(motifRC, f, listeSeqAlea)

#Recherche des motifs dans le liste des séquences implantées avec le motif complémentaire
motifsRCFound = searchMotifs(k, listeSeqMotifRC)

#Recherche des top motifs les plus fréquents
topMotifsRC = getTopMotifs(motifsRCFound, top)
print("Les top =", top, "motifs les plus fréquents de k =",k,"nucléotides trouvés sont :\n")
print(topMotifsRC)

Le motif implanté : TATCACCTC 

Les top = 2 motifs les plus fréquents de k = 9 nucléotides trouvés sont :

{'TATCACCTC': 5, 'CTATCACCT': 4}



## Partie B : Recherche de motifs identiques sur vos données

1\. Le fichier "C_glabrata_1000bp_only.fasta" contiens les séquences régulatrices pour tous les gènes de votre organisme. Nous avons pris les 1000bp en amont du codon start. Chercher les 50 motifs de taille 7 les plus fréquents dans les deux brim de votre génome.

In [7]:
k = 7 #Taille des motifs
top = 6 #Le nombre de motifs les plus fréquents à chercher
genome = "C_glabrata_1000bp_only.fasta"


def readFasta(fastaFileName):
    """
    Read a fasta file
    entrée fastaFileName: nom du fichier fasta
    sortie sequences: liste contenant toutes les sequences du fichier
    """
    sequence = []
    file = open(genome, "r")
    sequence = []
    for s in file:
        if s[0] != ">":
            sequence.append(s.strip().upper())
    return sequence

# Lecture du fichier fasta
sequences = readFasta(genome)


# RECHERCHE DE MOTIFS DANS LE BRIN SENS

print("Searching motifs...\n")
dico_motifs_sens = searchMotifs(k, sequences)

print("The",top,"most abundant motifs are \n")
print(getTopMotifs(dico_motifs_sens, top),"\n")


# RECHERCHE DE MOTIFS DANS LE BRIN COMPLÉMENTAIRE

# On récupère une liste du reverse complément de chaque séquence
sequences_rc = [reversecompl(sequence) for sequence in sequences]

print("Searching reverse complementary motifs...\n")
dico_motifs_antisens = searchMotifs(k, sequences_rc)


print("The", top, "most abundant motifs found in reverse complementary are \n")
print(getTopMotifs(dico_motifs_antisens, top),"\n")


# RECHERCHE DE MOTIFS DANS LE DICTIONNAIRE FUSIONNÉ

#copie du dictionnaire des motifs trouvés dans le brins sens
motifsData = copy.deepcopy(dico_motifs_sens)
#ajout des valeurs du dictionnaire des motifs antisens dans la copie
for clef, valeur in dico_motifs_antisens.items():
    if clef in motifsData :
        motifsData[clef] += valeur
    else : 
        motifsData[clef] = valeur

print("The",top,"most abundant motifs found are \n")
motifsTopData = getTopMotifs(motifsData, top)
print(motifsTopData)
 

Searching motifs...

The 6 most abundant motifs are 

{'AAAAAAA': 8385, 'TTTTTTT': 7658, 'ATATATA': 3096, 'ATTTTTT': 2970, 'AAAAAAT': 2960, 'TATATAT': 2796} 

Searching reverse complementary motifs...

The 6 most abundant motifs found in reverse complementary are 

{'TTTTTTT': 8385, 'AAAAAAA': 7658, 'TATATAT': 3096, 'AAAAAAT': 2970, 'ATTTTTT': 2960, 'ATATATA': 2796} 

The 6 most abundant motifs found are 

{'TTTTTTT': 16043, 'AAAAAAA': 16043, 'ATTTTTT': 5930, 'AAAAAAT': 5930, 'TATATAT': 5892, 'ATATATA': 5892}


2\. Quel sont les trois motifs de taille 7 les plus frequents? Pensez vous que ces motifs correspondent à de facteur de transcription connus? Justifier votre reponse.

<font color='blue'>
Reponse : Les motifs trouvés vont toujours par pairs. Chaque motif et son reverse complement sont trouvés le même nombre de fois. Ainsi les trois paires de motifs les plus fréquents sont ('TTTTTTT','AAAAAAA'), ('ATTTTTT','AAAAAAT') et ('TATATAT','ATATATA'). Et les trois motifs les plus fréquents sont : 'TTTTTTT', 'AAAAAAA' et 'ATTTTTT'.
</font>

3\. Le motif TGATTCAT correspond au facteur de transcription Gcn4 qui est très souvent trouvé dans le genome de levures. Est-ce que vous avez trouvé ce motif? Si oui avec quel frequence?

In [8]:
motifGcn4 = ['TGATTCA']

def searchGivenMotif(motifsTrouve, motifSpecifique):
    
    """
    Cherche une liste de motifs specifiques dans un dictionaire de motifs trouvés
    entrée motifsTrouve : dictionnaire de motifs, clé=motif, valeur = fréquence d'observation
    entrée motifSpecifique: liste de motifs specifiques à chercher
    sortie None : affiche le ranking des motifs especifique et sa frequence d'observation
    >>>searchGivenMotif({'TAA': 4, 'AAG': 3, 'AGT': 1}, ['AAG'])
    2 - AGG - 3
    """
    #Création d'une liste des (key, value) trié des motifsTrouve
    sortedDict = sorted(motifsTrouve.items(), key=lambda x: x[1], reverse = True)
    
    for motif in motifSpecifique : 
        
        if motif in motifsTrouve : #Le motif a été trouvé
            
            #On parcourt la liste pour récupérer le ranking du motif
            for i, (clef, valeur) in enumerate(sortedDict) :
                
                if clef == motif :
                    print(i+1,"-",motif,"-",motifsTrouve[motif])
                    break
                    
        else :
            print("Le motif", motif, "n'a pas été trouvé")
           
    return None

print ("\nMotifs dans le brin sens")
searchGivenMotif(dico_motifs_sens, motifGcn4)    
print ("\nMotifs dans le brin reverse complément")
searchGivenMotif(dico_motifs_antisens, motifGcn4)   




Motifs dans le brin sens
1819 - TGATTCA - 591

Motifs dans le brin reverse complément
1791 - TGATTCA - 594


4\. Les motifs peu complexes (avec par exemple 5, 6 ou 7 fois la même lettre) sont courants dans les génomes, ils n'ont généralement pas de signification biologique. Dans le context de ce projet, vous pouvez eliminer aussi les motifis ayant deux lettre repetés, comme par exemple AGAGAGA. Faites une fonction pour éliminer les motifs peu complexes, _i. e._ qui ont au moins $m$ fois la même lettre ou qui ont deux lettre consecutive répétés. Dans quel position vous trouver Gcn4 apres enlever les motifs peu complexes?</font>. 

In [9]:
# On retire les motifs avec au moins reLow fois la même lettre et des motifs de deux lettre répétés trois fois

def removeLowComplexe(motifs, reLow = 5):
    """
    Eleve les motifs peu complexe 
    entrée motifs: dictionnaire de motifs, clé=motif, valeur = fréquence d'observation
    sortie validMotif: dictionnaire de motifs filtré, clé=motif, valeur = fréquence d'observation
    """
    validMotif = {}
    
    for clef, val in motifs.items() : 
        
        cinq_fois_même_lettre = '([ACTG]*A){'+ str(reLow) +'}|([ACTG]*C){'+ str(reLow) +'}|([ACTG]*T){'+ str(reLow) +'}|([ACTG]*G){'+ str(reLow) +'}'
        deux_lettres_repetee = '([ACTG]*AC){3}|([ATCG]*AT){3}|([ATCG]*AG){3}|([ATCG]*TA){3}|([ATCG]*TG){3}|([ATCG]*TC){3}|([ATCG]*CG){3}|([ATCG]*CA){3}|([ATCG]*CT){3}|([ATCG]*GA){3}|([ATCG]*GT){3}|([ATCG]*GC){3}'
        
        p = re.compile(cinq_fois_même_lettre+deux_lettres_repetee)
        m = p.match(clef)
        
        if not m:
            validMotif[clef] = val
            
    return validMotif



motifsFiltreData = removeLowComplexe(dico_motifs_sens)
motifsFiltreReverse = removeLowComplexe(dico_motifs_antisens)

print ("\nPosition de",motifGcn4,"dans le brin sens")
searchGivenMotif(motifsFiltreData, motifGcn4)    
print ("\nPosition de",motifGcn4,"dans le brin reverse complément")
searchGivenMotif(motifsFiltreReverse, motifGcn4)   



Position de ['TGATTCA'] dans le brin sens
1356 - TGATTCA - 591

Position de ['TGATTCA'] dans le brin reverse complément
1330 - TGATTCA - 594


5\. Le fichier "Sequence_by_Peaks_G*.fasta" contiens les regions de peak trouvé par ChipSeq, qui contient probablement le Facteur de Transcription que nous cherchons. Apliquer les fonctions précédents pour chercher les 3 motifs les plus fréquents dans les deux brins. Il faut bien evidemment enlever les motifs peu complexe.

In [10]:
k = 7
top = 6
reLow = 5

genome = "../Sequence_by_Peaks_1.fasta"

sequences = readFasta(genome)

print("Searching fwd motifs...\n")
dico_motifs_sens_projet = searchMotifs(k, sequences)
# On récupère une liste du reverse complément de chaque séquence
sequences_rc = [reversecompl(sequence) for sequence in sequences]
dico_motifs_antisens_projet = searchMotifs(k, sequences_rc)

print("The",top,"most abundant motifs are \n")
print(getTopMotifs(dico_motifs_sens_projet, top),"\n")

print("The",top,"most abundant reverse compl motifs are \n")
print(getTopMotifs(dico_motifs_antisens_projet, top),"\n")


#On retire les motifs de faible complexité
dico_motifs_sens_projet_filtre = removeLowComplexe(dico_motifs_sens_projet, reLow)
dico_motifs_antisens_projet_filtre = removeLowComplexe(dico_motifs_antisens_projet, reLow)

print("The",top,"most abundant no low-complexe motifs are \n")
print(getTopMotifs(dico_motifs_sens_projet_filtre, top),"\n")

print("The",top,"most abundant no low-complexe motifs in reverse compl are \n")
print(getTopMotifs(dico_motifs_antisens_projet_filtre, top),"\n")

# RECHERCHE DE MOTIFS DANS LE DICTIONNAIRE FUSIONNÉ

#copie du dictionnaire des motifs trouvés dans le brins sens
motifsData_projet_filtre = copy.deepcopy(dico_motifs_sens_projet_filtre)

#ajout des valeurs du dictionnaire des motifs antisens dans la copie
for clef, valeur in dico_motifs_antisens_projet_filtre.items():
    if clef in motifsData_projet_filtre :
        motifsData_projet_filtre[clef] += valeur
    else : 
        motifsData_projet_filtre[clef] = valeur
        
print("The",top,"most abundant no low-complexe motifs are \n")
print(getTopMotifs(motifsData_projet_filtre, top),"\n")

Searching fwd motifs...

The 6 most abundant motifs are 

{'AAAAAAA': 42, 'AGCAAAA': 26, 'TATTATT': 23, 'GCAAAAA': 22, 'TTATTAT': 21, 'CAAAAAT': 21} 

The 6 most abundant reverse compl motifs are 

{'TTTTTTT': 42, 'TTTTGCT': 26, 'AATAATA': 23, 'TTTTTGC': 22, 'ATAATAA': 21, 'ATTTTTG': 21} 

The 6 most abundant no low-complexe motifs are 

{'ATTATTA': 20, 'CAGCAAA': 19, 'TTATCAT': 17, 'ATCATTT': 17, 'AAATATT': 17, 'AAAATTT': 16} 

The 6 most abundant no low-complexe motifs in reverse compl are 

{'TAATAAT': 20, 'TTTGCTG': 19, 'ATGATAA': 17, 'AAATGAT': 17, 'AATATTT': 17, 'AAATTTT': 16} 

The 6 most abundant no low-complexe motifs are 

{'TTTGCTG': 32, 'CAGCAAA': 32, 'AAAATTT': 27, 'AAATTTT': 27, 'ATTATTA': 24, 'ATATTTA': 24} 



<font color = blue>
1e motif le plus fréquent et son reverse complément : TATTGCTG et CAGCAATA
   
"CAGCAA(TA)" -> Amt1 "CAGCAA(AA)
  
2e motif le plus fréquent et son reverse complément : ATTATTGC et GCAATAAT
   
On retrouve "GCAA(TA)"

On remarque alors que le motif CAGCAAAA possède 5 fois la même lettre, il a alors été retiré de notre set de motif recherché. Nous réeffectuons l'opération avec reLow = 6 pour garder ce motif en cherchant des motifs de taille 8.

</font>

In [11]:
k = 8
top = 6
reLow = 6

genome = "../Sequence_by_Peaks_1.fasta"

sequences = readFasta(genome)

print("Searching fwd motifs...\n")
dico_motifs_sens_projet = searchMotifs(k, sequences)
# On récupère une liste du reverse complément de chaque séquence
sequences_rc = [reversecompl(sequence) for sequence in sequences]
dico_motifs_antisens_projet = searchMotifs(k, sequences_rc)

print("The",top,"most abundant motifs are \n")
print(getTopMotifs(dico_motifs_sens_projet, top),"\n")

print("The",top,"most abundant reverse compl motifs are \n")
print(getTopMotifs(dico_motifs_antisens_projet, top),"\n")


#On retire les motifs de faible complexité
dico_motifs_sens_projet_filtre = removeLowComplexe(dico_motifs_sens_projet, reLow)
dico_motifs_antisens_projet_filtre = removeLowComplexe(dico_motifs_antisens_projet, reLow)

print("The",top,"most abundant no low-complexe motifs are \n")
print(getTopMotifs(dico_motifs_sens_projet_filtre, top),"\n")

print("The",top,"most abundant no low-complexe motifs in reverse compl are \n")
print(getTopMotifs(dico_motifs_antisens_projet_filtre, top),"\n")

# RECHERCHE DE MOTIFS DANS LE DICTIONNAIRE FUSIONNÉ

#copie du dictionnaire des motifs trouvés dans le brins sens
motifsData_projet_filtre = copy.deepcopy(dico_motifs_sens_projet_filtre)

#ajout des valeurs du dictionnaire des motifs antisens dans la copie
for clef, valeur in dico_motifs_antisens_projet_filtre.items():
    if clef in motifsData_projet_filtre :
        motifsData_projet_filtre[clef] += valeur
    else : 
        motifsData_projet_filtre[clef] = valeur
        
print("The",top,"most abundant no low-complexe motifs are \n")
print(getTopMotifs(motifsData_projet_filtre, top),"\n")

Searching fwd motifs...

The 6 most abundant motifs are 

{'AGCAAAAA': 20, 'AAAAAAAA': 19, 'CAGCAAAA': 18, 'TTATTATT': 15, 'TTTTGCTG': 11, 'TAAAAAAA': 11} 

The 6 most abundant reverse compl motifs are 

{'TTTTTGCT': 20, 'TTTTTTTT': 19, 'TTTTGCTG': 18, 'AATAATAA': 15, 'CAGCAAAA': 11, 'TTTTTTTA': 11} 

The 6 most abundant no low-complexe motifs are 

{'CAGCAAAA': 18, 'TTTTGCTG': 11, 'GCAAAAAT': 11, 'CAGCAATA': 10, 'AACAGCAA': 10, 'ATTTTCAT': 9} 

The 6 most abundant no low-complexe motifs in reverse compl are 

{'TTTTGCTG': 18, 'CAGCAAAA': 11, 'ATTTTTGC': 11, 'TATTGCTG': 10, 'TTGCTGTT': 10, 'ATGAAAAT': 9} 

The 6 most abundant no low-complexe motifs are 

{'TTTTGCTG': 29, 'CAGCAAAA': 29, 'ATTTTTGC': 16, 'GCAAAAAT': 16, 'TATTGCTG': 13, 'CAGCAATA': 13} 



<font color = blue>
    Le motif "CAGCAAAA" est le motif le plus fréquent (avec son reverse complément "TTTTGCTG"). Cela confirme notre hypothèse à propos du facteur de transcription Amt1. Le deuxième motif le plus fréquent avec 16 occurence est GCAAAAAT (avec son reverse complément : ATTTTTGC). On retrouve le motif GCAAAA qui fait partie de notre séquence consensus. Le troisième motif le plus fréquent est le motif CAGCAATA (avec son reverse complément CAGCAATA) qui correspond à notre motif avec une variation à la position 7 avec la substitution A -> T
</font>

6\. Ulitser la base YEASTRACT : http://www.yeastract.com/formsearchbydnamotif.php pour chercher les motifs. 
Avez vous une indication pour le facteur de transcription impliqué ?


<font : color = 'blue'>
Les motifs les plus abondants retrouvés dans le fichier "Sequence_by_Peaks_1.fasta" ont la plus grande probabilité de se raprocher exactement de motifs nucléotidiques présents dans le portion d'ADN responsable de la fixation du facteur de transcription étudié. En cherchant le motif le plus fréquent trouvé : CAGCAAAA, on ne trouve aucun facteur de transcription correspondant dans la base YEASTRACT.
</font>